## Testing Cassandra
En este notebook se realizarán las pruebas de conexión y inserción de datos en Cassandra así como las consultas de datos.
<div align="center">
    <img src="img/initial_table.png" width="500">
</div>

In [22]:
import pandas as pd
from cassandra.cluster import Cluster
from cassandra.policies import DCAwareRoundRobinPolicy
import time
import random
import logging
from faker import Faker

In [17]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [18]:
# Función para medir el tiempo de ejecución de una función
def measure_request_time(func, *args, **kwargs):
    """Mide el tiempo de ejecución de una función."""
    start_time = time.time()  # Iniciar el temporizador
    result = func(*args, **kwargs)  # Ejecutar la función
    end_time = time.time()  # Finalizar el temporizador
    duration = end_time - start_time  # Calcular la duración
    print(f"Tiempo de ejecución: {duration:.4f} segundos")
    return result


In [23]:
# Inicializar Faker
fake = Faker()

# Función para generar un nombre aleatorio
def generate_random_name():
    return fake.first_name()

# Función para generar un apellido aleatorio
def generate_random_last_name():
    return fake.last_name()

# Función para generar un email aleatorio
def generate_random_email():
    return fake.email()

# Función para generar un género aleatorio
def generate_random_gender():
    return random.choice(['Male', 'Female', 'Other'])

# Generar 1 millón de registros
data = {
    'id': [],
    'first_name': [],
    'last_name': [],
    'email': [],
    'gender': []
}

for i in range(22, 100001):
    data['id'].append(i)
    data['first_name'].append(generate_random_name())
    data['last_name'].append(generate_random_last_name())
    data['email'].append(generate_random_email())
    data['gender'].append(generate_random_gender())
    
    # Loguear cada 100,000 registros
    if i % 1000000 == 0:
        logging.info(f"Generados {i} registros")

# Crear DataFrame
df = pd.DataFrame(data)

# Guardar DataFrame en un archivo CSV
df.to_csv('/data/MOCK_DATA_LONG.csv', index=False)

logging.info("Archivo CSV generado correctamente con 1 millón de registros.")

2024-11-13 17:54:56,877 - INFO - Archivo CSV generado correctamente con 1 millón de registros.


In [24]:
def wait_for_cassandra():
    while True:
        try:
            cluster = Cluster(['cassandra'], load_balancing_policy=DCAwareRoundRobinPolicy(), protocol_version=3)
            session = cluster.connect()
            return cluster, session
        except Exception as e:
            print(f"Espere por favor... Se está cargando Cassandra: {e}")
            time.sleep(20)

In [25]:
def insert_data_from_csv(session, file_path):
    try:
        # Leer datos del archivo CSV
        df = pd.read_csv(f'{file_path}')
        print("Archivo CSV leído correctamente")
    
        # Insertar datos desde el DataFrame
        for index, row in df.iterrows():
            session.execute("""
                INSERT INTO user_data.users (id, first_name, last_name, email, gender)
                VALUES (%s, %s, %s, %s, %s)
            """, (row['id'], row['first_name'], row['last_name'], row['email'], row['gender']))
    
        print("Datos insertados correctamente")

    except Exception as e:
        print(f"Error al procesar el archivo CSV: {e}")

cluster, session = wait_for_cassandra()

/tmp/ipykernel_32/508461982.py:4: DeprecationWarning: Legacy execution parameters will be removed in 4.0. Consider using execution profiles.
  cluster = Cluster(['cassandra'], load_balancing_policy=DCAwareRoundRobinPolicy(), protocol_version=3)
2024-11-13 17:56:00,599 - INFO - Using datacenter 'datacenter1' for DCAwareRoundRobinPolicy (via host '172.23.0.2:9042'); if incorrect, please specify a local_dc to the constructor, or limit contact points to local cluster nodes


In [26]:
def long_data_insert(session):
    insert_data_from_csv(session, '/data/MOCK_DATA_LONG.csv')
    rows = session.execute("SELECT * FROM user_data.users")
    # Convertir ResultSet a DataFrame
    df_rows = pd.DataFrame(list(rows))
    print(df_rows.head())

measure_request_time(long_data_insert, session)

Archivo CSV leído correctamente


2024-11-13 17:56:18,737 - WARNING - Clock skew detected: current tick (1731520578737474) was 1912930 microseconds behind the last generated timestamp (1731520580650404), returned timestamps will be artificially incremented to guarantee monotonicity.
2024-11-13 17:56:46,747 - WARNING - Clock skew detected: current tick (1731520606747223) was 1952932 microseconds behind the last generated timestamp (1731520608700155), returned timestamps will be artificially incremented to guarantee monotonicity.


Datos insertados correctamente
      id                      email first_name  gender last_name
0   4317      davisdana@example.org      Terri  Female      Shaw
1  62693          jfrey@example.net    Stephen   Other   Jackson
2  51678       daniel46@example.com  Elizabeth    Male     Reyes
3  77328   schmidtjames@example.com    Kristen  Female      Cain
4  84757  prestonthomas@example.org     Steven  Female    Horton
Tiempo de ejecución: 69.2137 segundos


In [ ]:
def fetch_all_users(session):
    """Recupera todos los usuarios de la base de datos y los muestra en una tabla estilo Excel."""
    try:
        # Ejecutar la consulta para obtener todos los usuarios
        rows = session.execute("SELECT * FROM user_data.users")
        
        # Convertir los resultados a un DataFrame
        users_df = pd.DataFrame(rows)
        
        # Mostrar los datos en formato estilo Excel
        print("\nDatos de usuarios:")
        styled_df = users_df.style.set_table_attributes('style="width:100%; border-collapse:collapse;"') \
                                   .set_table_styles([{'selector': 'th', 'props': [('background-color', '#f2f2f2'), ('font-weight', 'bold')]}]) \
                                   .set_properties(**{'border': '1px solid black'}) \
        
        # Mostrar el DataFrame estilizado
        display(styled_df)
        
    except Exception as e:
        print(f"Error al recuperar datos: {e}")

# Intentar ejecutar la acción de recuperación de usuarios
cluster, session = wait_for_cassandra()
fetch_all_users(session)


In [27]:
def get_user_by_id(session, user_id):
    """Recupera un usuario por su ID desde la base de datos."""
    try:
        result = session.execute("""
            SELECT * FROM user_data.users WHERE id = %s
        """, (user_id,))
        
        if result:
            for row in result:
                print(f"ID: {row.id}, Nombre: {row.first_name} {row.last_name}, Email: {row.email}, Género: {row.gender}")
        else:
            print("Usuario no encontrado.")
    
    except Exception as e:
        print(f"Error al recuperar el usuario: {e}")


In [28]:
def update_user_email(session, user_id, new_email):
    try:
        session.execute("""
            UPDATE user_data.users
            SET email = %s
            WHERE id = %s
        """, (new_email, user_id))
        print(f"Email actualizado para el usuario con ID {user_id}.")
    except Exception as e:
        print(f"Error al actualizar el email: {e}")


cluster, session = wait_for_cassandra()

measure_request_time(update_user_email, session, 1, "vegetta_email@example.com")
measure_request_time(get_user_by_id, session, 1)

/tmp/ipykernel_32/508461982.py:4: DeprecationWarning: Legacy execution parameters will be removed in 4.0. Consider using execution profiles.
  cluster = Cluster(['cassandra'], load_balancing_policy=DCAwareRoundRobinPolicy(), protocol_version=3)
2024-11-13 17:57:44,628 - INFO - Using datacenter 'datacenter1' for DCAwareRoundRobinPolicy (via host '172.23.0.2:9042'); if incorrect, please specify a local_dc to the constructor, or limit contact points to local cluster nodes


Email actualizado para el usuario con ID 1.
Tiempo de ejecución: 0.0035 segundos
ID: 1, Nombre: Athena Elijahu, Email: vegetta_email@example.com, Género: Female
Tiempo de ejecución: 0.0061 segundos


In [29]:
def get_emails(session):
    """Recupera los correos electrónicos de los usuarios desde la base de datos."""
    try:
        rows = session.execute("SELECT email FROM user_data.users LIMIT 20")
        emails = [row.email for row in rows]
        for email in emails:
            print(email)
    
    except Exception as e:
        print(f"Error al ejecutar la función: {e}")
        return []

measure_request_time(get_emails, session)

davisdana@example.org
jfrey@example.net
daniel46@example.com
schmidtjames@example.com
prestonthomas@example.org
shelly24@example.net
linedward@example.org
uvincent@example.org
davissarah@example.net
joanchang@example.com
bbaldwin@example.com
barbaraperry@example.org
bennettandrew@example.com
gbarton@example.net
evansjessica@example.com
dunlapkendra@example.net
mikekennedy@example.org
kristinalee@example.net
monica64@example.com
stephanie16@example.net
gary73@example.com
tmcdonald@example.com
leroy82@example.org
shannonmorris@example.net
ryan81@example.net
vwalters@example.com
bflores@example.com
christinahines@example.org
qbarry@example.net
sjoyce@example.org
jessica56@example.com
jenkinskathy@example.com
perezcrystal@example.net
dnelson@example.org
istone@example.net
lisa46@example.net
kimgonzalez@example.net
tonyahunter@example.org
vmcdowell@example.com
oharris@example.org
omontoya@example.org
chelsea60@example.com
julie24@example.org
william27@example.net
amandacollins@example.net
m